<a href="https://colab.research.google.com/github/Meta-user-byte/MIT-iQuHack/blob/main/quantum_rings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install QuantumRingsLib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.5 MB/s eta 0:00:00


In [2]:
!pip install qiskit==1.3.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 10.3 MB/s eta 0:00:00


In [3]:
import qiskit
import QuantumRingsLib
print (QuantumRingsLib.__version__)

0.9.0


In [6]:
!pip install quantumrings-toolkit-qiskit


In [9]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np


provider = QuantumRingsProvider(
    token='rings-200.S6LCitPyRz7XDlPjsqk9Y3KFUUxtbtLY',
    name='ankitphysics5327@rajdhani.du.ac.in'
)
backend = provider.get_backend("scarlet_quantum_rings")

provider.active_account()

{'name': 'ankitphysics5327@rajdhani.du.ac.in',
 'token': 'rings-200.S6LCitPyRz7XDlPjsqk9Y3KFUUxtbtLY',
 'max_qubits': '200'}

In [10]:
from QuantumRingsLib import Parameter, ParameterVector
import math

In [11]:
import csv
from concurrent.futures import ThreadPoolExecutor

In [12]:
def approximate_iqft(qc, qubits, precision_cutoff=2):
    """
    Approximate Inverse QFT with reduced gate count.
    """
    n = len(qubits)
    for i in range(n):
        qc.h(qubits[i])
        for j in range(1, min(i, precision_cutoff) + 1):
            qc.cu1(-np.pi / (2 ** (i - j + 1)), qubits[j - 1], qubits[i])
    qc.barrier()


In [13]:
def modular_exponentiation_optimized(qc, control_qubits, target_qubits, a, N):
    """
    Optimized modular exponentiation with reduced gate depth.
    """
    n = len(control_qubits)
    for i in range(n):
        exponent = 2 ** i
        mod_exp = pow(a, exponent, N)
        if mod_exp != 1:
            # Apply optimized controlled operations
            for j in range(len(target_qubits)):
                if (mod_exp >> j) & 1:
                    qc.cx(control_qubits[i], target_qubits[j])  # Keep CNOTs minimal
    qc.barrier()


In [14]:
def plot_histogram(counts, title=''):
    fig, ax = plt.subplots(figsize=(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]
    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)
    plt.bar(unique, bin_counts)
    plt.title(title)
    plt.show()


In [15]:
def initialize_circuit(bit_size):
    control_qubits_count = bit_size + 2  # Add extra qubits for precision
    num_qubits = control_qubits_count + bit_size // 2 + 2

    q = QuantumRegister(num_qubits, 'q')
    c = ClassicalRegister(control_qubits_count, 'c')
    qc = QuantumCircuit(q, c)

    return qc, q, c



In [16]:
# Superposition Creation
def create_superposition(qc, control_qubits):
    for qubit in control_qubits:
        qc.h(qubit)
    qc.barrier()


In [17]:
import time

In [18]:
# Measurement and Execution with Timing
def measure_and_execute(qc, control_qubits, classical_bits, shots=10000):
    start_time = time.time()
    for i in range(min(len(control_qubits), len(classical_bits))):
        qc.measure(control_qubits[i], classical_bits[i])
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    execution_time = time.time() - start_time
    return counts, execution_time



In [5]:
from semiprimes import semiprimes

In [4]:
# Shor's Algorithm
def shors_algorithm(bit_size, a=7):
    N = semiprimes[bit_size]
    qc, q, c = initialize_circuit(bit_size)
    control_qubits = [q[i] for i in range(bit_size)]
    target_qubits = [q[j] for j in range(bit_size, len(q))]

    create_superposition(qc, control_qubits)
    qc.x(len(q) - 1)
    modular_exponentiation_optimized(qc, control_qubits, target_qubits, a, N)

    counts, execution_time = measure_and_execute(qc, control_qubits, c)
    most_common_state = max(counts, key=counts.get)
    period_guess = int(most_common_state, 2)
    f1, f2 = find_factors(N, a, period_guess)

    if f1 and f2:
        print(f"Factors of {N}: {f1} and {f2} in {execution_time:.2f} seconds.")
    else:
        print(f"Failed to factor {N}, rerun the quantum algorithm.")

    return counts, execution_time, N, a

# New Section

In [6]:
semiprimes[8]

143

In [7]:
from concurrent.futures import ThreadPoolExecutor, as_completed, TimeoutError

# Parallel Execution for Multiple Semiprimes
def log_results():
    bit_sizes = [8, 10, 12, 14, 16, 24, 32]
    logs = []

    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(shors_algorithm, bit_size) for bit_size in bit_sizes]

        for future in futures:
            counts, execution_time, N, a = future.result()
            most_common_state = max(counts, key=counts.get)
            period_guess = int(most_common_state, 2)
            f1, f2 = find_factors(N, a, period_guess)

            logs.append({
                "Bit Size": bit_size,
                "Semiprime": N,
                "Factors": f"{f1}, {f2}" if f1 and f2 else "Failed",
                "Execution Time (s)": execution_time
            })

    return logs

In [ ]:
# Save Logs to CSV
def save_logs_to_csv(logs, filename="shors_results.csv"):
    with open(filename, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=logs[0].keys())
        writer.writeheader()
        writer.writerows(logs)

# Run the Logging and Save Results
logs = log_results()
save_logs_to_csv(logs)

In [ ]:
from math import gcd

def find_factors(N, a, r):
    # Step 1: Ensure r is valid
    if r % 2 != 0:
        return None, None  # Period must be even

    # Step 2: Compute the GCDs
    factor1 = gcd(pow(a, r // 2) - 1, N)
    factor2 = gcd(pow(a, r // 2) + 1, N)

    # Step 3: Check if factors are non-trivial
    if factor1 in [1, N] or factor2 in [1, N]:
        return None, None  # Trivial factors found, rerun the quantum part

    return factor1, factor2



In [ ]:
from math import gcd

# Try different values of a
for a in range(2, N):
    if gcd(a, N) != 1:
        continue  # Skip if not coprime with N

    # Assume period found from quantum circuit
    r = 44  # Placeholder; you'd get this from actual measurements

    if pow(a, r, N) != 1 or r % 2 != 0:
        continue  # Skip if period is invalid

    f1, f2 = find_factors(N, a, r)
    if f1 and f2:
        print(f"Factors of {N} are: {f1} and {f2}")
        break
else:
    print("Failed with all bases. Rerun the quantum algorithm.")


